In this notebook we are comparing how different PCA dimensions values are affecting the performance of model `MLPRegressor`. This model is a neural network model selected in notebook `02. Training ML models`.

At the same time we have given more time to the model, hand adjusted subset of the params and applied an invascaling strategy of decresing the learing rate in time to fit the model more precisely.

In [1]:
LOWER_LIMIT = -7

In [2]:
#loading dataset
import numpy as np

input_and_output = np.load("../final/dataset.npz")
inputs  = input_and_output["inputs"].astype(np.float64)
outputs = input_and_output["outputs"].astype(np.float64)
dataset_size = inputs.shape[0]

print("loaded dataset")

loaded dataset


In [4]:
# dropping treatment column in input
def drop_treatment(input_data: np.ndarray) -> np.ndarray:
    """Drops treatment data from the dataset"""
    if input_data.shape[1] == 11:
        return input_data[:, 1:]

    return input_data

input_without_treatment = drop_treatment(inputs)
print("dropped treatment column")


def output_transform(outputs: np.array) -> np.array:
    x = np.copy(outputs)
    zeros_in_output = x <= 0
    x[zeros_in_output] = 1
    y = np.log10(x)
    y[zeros_in_output] = LOWER_LIMIT
    y[y < LOWER_LIMIT] = LOWER_LIMIT
    return y
    
def output_untransform(transformed_outputs: np.array) -> np.array:
    lower_limits = transformed_outputs < LOWER_LIMIT
    z = 10 ** transformed_outputs
    z[lower_limits] = 0
    return z

def apply_size_limit(outputs: np.array) -> np.array:
    x = np.copy(outputs)
    x[x < LOWER_LIMIT] = LOWER_LIMIT
    return x

def apply_absolute_size_limit(outputs: np.array) -> np.array:
    limit = 10 ** LOWER_LIMIT
    x = np.copy(outputs)
    x[x < limit] = 0
    return x

outputs_order_of_magnitude = output_transform(outputs)
print("transformed to orders of magnitude")

#splitting data into train, test, validate datasets 
train_size = int(dataset_size * 0.7)
test_size = int(dataset_size * 0.15)

X_train = input_without_treatment[:train_size, :]
Y_train_absolute = apply_absolute_size_limit(outputs[:train_size, :])
X_test = input_without_treatment[train_size:(train_size + test_size), :]
Y_test_absolute = apply_absolute_size_limit(outputs[train_size:(train_size + test_size), :])
Y_train = outputs_order_of_magnitude[:train_size, :]
Y_test = outputs_order_of_magnitude[train_size:(train_size + test_size), :]

# scaling inputs

import pickle
from pathlib import Path

LOGNORMAL_PARAMETERS = (1, 2)

class CustomScaler:
    def __init__(self):
        super().__init__()
        self.scaler = MinMaxScaler()
        self.plot_loval = [0.0] * len(LOGNORMAL_PARAMETERS)
        self.plot_hival = [1.0] * len(LOGNORMAL_PARAMETERS)

    def transform(self, x: np.ndarray, copy=None) -> np.ndarray:
        res = self.scaler.transform(x)
        for i, parameter_index in enumerate(LOGNORMAL_PARAMETERS):
            res[:, parameter_index] = (x[:, parameter_index] - self.plot_loval[i]) / (self.plot_hival[i] - self.plot_loval[i])

        return res

    def fit(self, x, copy=None):
        self.scaler.fit(x)
        for i, parameter_index in enumerate(LOGNORMAL_PARAMETERS):
            column_values = x[:, parameter_index]

            quantile_1, quantile_3 = np.quantile(column_values, [0.25, 0.75], axis=0)
            iqr = quantile_3 - quantile_1

            loval = quantile_1 - 1.5 * iqr
            hival = quantile_3 + 1.5 * iqr

            wiskhi = np.compress(column_values <= hival, column_values)
            wisklo = np.compress(column_values >= loval, column_values)
            actual_hival = np.max(wiskhi)
            actual_loval = np.min(wisklo)

            self.plot_loval[i] = actual_loval
            self.plot_hival[i] = actual_hival

        return self

    def inverse_transform(self, x, copy=None):
        res = self.scaler.inverse_transform(x)
        for i, parameter_index in enumerate(LOGNORMAL_PARAMETERS):
            res[:, parameter_index] = x[:, parameter_index] * (self.plot_hival[i] - self.plot_loval[i]) + self.plot_loval[i]
        return res

scaler_path = Path(f"../final/scaler.pickle")
scaler = None
if scaler_path.exists():
    with scaler_path.open("rb") as scaler_file:
        scaler = pickle.load(scaler_file)
else:
    scaler = CustomScaler().fit(X_train)
    with scaler_path.open("wb") as opened_file:
        pickle.dump(scaler, opened_file)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("scaled")

iterations = 7

Path("../final/pca_models").mkdir(parents=True, exist_ok=True)
for PCA_COMPONENTS in [8, 10, 11, 12, 13, 16, 20]:
    # applying principal component analysis
    from sklearn.decomposition import PCA

    pca_path = Path(f"../final/pca/{PCA_COMPONENTS}_{LOWER_LIMIT}.pickle")

    if pca_path.exists():
        with pca_path.open("rb") as opened_file:
            pca = pickle.load(opened_file)
        Y_train_pca = pca.transform(Y_train)
    else: 
        pca = PCA(n_components=PCA_COMPONENTS)
        Y_train_pca = pca.fit_transform(Y_train)
        with pca_path.open("wb") as opened_file:
            pickle.dump(pca, opened_file)

    from functools import reduce
    print(f"applied pca with {PCA_COMPONENTS} components. Unexplained variance ratio: {reduce(lambda a, b: a - b, pca.explained_variance_ratio_, 1.0)}")

    import time
    from sklearn.metrics import mean_squared_error

    from sklearn.neural_network import MLPRegressor
    from threadpoolctl import threadpool_limits
    from cpuinfo import get_cpu_info
    import json

    hidden_layer_sizes = [600, 100, 40]
    training_start = time.time()
    for k in range(iterations):
        last_file = f"../final/pca_models/MLPRegressor_{'_'.join(str(i) for i in hidden_layer_sizes)}_{PCA_COMPONENTS}_{LOWER_LIMIT}_{k}.pickle"
        info_filename = f"../final/pca_models/MLPRegressor_{'_'.join(str(i) for i in hidden_layer_sizes)}_{PCA_COMPONENTS}_{LOWER_LIMIT}_{k}.json"

        if Path(last_file).exists():
            print(f"loading previous {last_file}")
            with Path(last_file).open("rb") as opened_file:
                model = pickle.load(opened_file)
            with Path(info_filename).open('r') as opened_file:
                print(opened_file.read())
            continue

        if k > 0:
            old_model = model
        model_params = {
            "alpha": 0.0040005316095293 / (2 ** k),
            "batch_size": 2000,
            "hidden_layer_sizes": hidden_layer_sizes,
            "learning_rate": "constant",
            "learning_rate_init": 0.00016798744315656234 / (2 ** k),
            "max_iter": 400,
            "n_iter_no_change": 5,
            "random_state": 42,
            "tol": 1e-05 / (2**k),
            "epsilon": 1e-08 / (2**k),
            "verbose": True,
            "warm_start": k > 0
        }
        model = MLPRegressor(**model_params)
        if k > 0:
            for variable_name in ("coefs_", "t_", "n_outputs_", "n_layers_", "out_activation_", "intercepts_", "n_iter_", "loss_curve_", "best_loss_", "_no_improvement_count"):
                setattr(model, variable_name, getattr(old_model, variable_name))

        with threadpool_limits(limits=get_cpu_info()["count"], user_api='blas'):
            start_time = time.time()
            model.fit(X_train_scaled, Y_train_pca)
            training_time_s = int(time.time() - start_time)
            
            start_time = time.time()
            test_result = pca.inverse_transform(model.predict(X_test_scaled))
            error_test  = mean_squared_error(Y_test,  apply_size_limit(test_result))
            error_test_absolute  = mean_squared_error(Y_test_absolute,  output_untransform(test_result))
            test_evaluation_s = int(time.time() - start_time)

            start_time = time.time()
            train_result = pca.inverse_transform(model.predict(X_train_scaled))
            error_train = mean_squared_error(Y_train, apply_size_limit(train_result))
            error_train_absolute = mean_squared_error(Y_train_absolute, output_untransform(train_result))
            train_evaluation_s = int(time.time() - start_time)

        print(f"error test: {error_test}, error train: {error_train} training_time: {time.time() - training_start:.1f}")

        with Path(last_file).open("wb") as opened_file:
            print(f"saving {last_file}")
            pickle.dump(model, opened_file)
        with Path(info_filename).open('w') as opened_file:
            info = json.dumps({
                "cpu_info": {key: get_cpu_info()[key] for key in ["arch", "bits", "brand_raw", "count", "l2_cache_size"]},
                "pca_components": PCA_COMPONENTS,
                "pca_unexplained_variance_ratio": reduce(lambda a, b: a - b, pca.explained_variance_ratio_, 1.0),
                "tumour_lower_size_limit_l": 10 ** LOWER_LIMIT,
                "tumour_lower_size_limit_log10_l": LOWER_LIMIT,
                "model_params": model_params,
                "test_dataset": "[700000:850000] of ../final/dataset.npz",
                "test_error_orders_of_magnitude": error_test,
                "test_error_absolute": error_test_absolute,
                "train_dataset": "[:700000] of ../final/dataset.npz",
                "train_error_orders_of_magnitude": error_train,
                "train_error_absolute": error_train_absolute,
                "training_time_s": training_time_s,
                "train_evaluation_s": train_evaluation_s,
                "test_evaluation_s": test_evaluation_s
            }, sort_keys=True, indent=4)
            print(f"saving info to file: {info_filename} {info}")
            opened_file.write(info)

dropped treatment column
transformed to orders of magnitude
scaled
applied pca with 8 components. Unexplained variance ratio: 1.841504622877898e-05
loading previous ../final/pca_models/MLPRegressor_600_100_40_8_-7_0.pickle

Iteration 146, loss = 0.00182458
Iteration 147, loss = 0.00151798
Iteration 148, loss = 0.00152546
Iteration 149, loss = 0.00152646
Iteration 150, loss = 0.00155442
Iteration 151, loss = 0.00154049
Iteration 152, loss = 0.00154396
Iteration 153, loss = 0.00153942
Training loss did not improve more than tol=0.000005 for 5 consecutive epochs. Stopping.
error test: 6.066106451667655e-05, error train: 5.779183179228414e-05 training_time: 328.1
saving ../final/pca_models/MLPRegressor_600_100_40_8_-7_1.pickle
saving info to file: ../final/pca_models/MLPRegressor_600_100_40_8_-7_1.json {
    "cpu_info": {
        "arch": "X86_64",
        "bits": 64,
        "brand_raw": "Intel(R) Core(TM) i7-4980HQ CPU @ 2.80GHz",
        "count": 8,
        "l2_cache_size": 262144
    },

error test: 4.698716297157301e-05, error train: 4.433434659605816e-05 training_time: 4975.3
saving ../final/pca_models/MLPRegressor_600_100_40_8_-7_3.pickle
saving info to file: ../final/pca_models/MLPRegressor_600_100_40_8_-7_3.json {
    "cpu_info": {
        "arch": "X86_64",
        "bits": 64,
        "brand_raw": "Intel(R) Core(TM) i7-4980HQ CPU @ 2.80GHz",
        "count": 8,
        "l2_cache_size": 262144
    },
    "model_params": {
        "alpha": 0.0005000664511911625,
        "batch_size": 2000,
        "epsilon": 1.25e-09,
        "hidden_layer_sizes": [
            600,
            100,
            40
        ],
        "learning_rate": "constant",
        "learning_rate_init": 2.0998430394570292e-05,
        "max_iter": 400,
        "n_iter_no_change": 5,
        "random_state": 42,
        "tol": 1.25e-06,
        "verbose": true,
        "warm_start": true
    },
    "pca_components": 8,
    "pca_unexplained_variance_ratio": 1.841504622877898e-05,
    "test_dataset":

/Users/tpawlowski/workspace/dokstud/lazarou2020/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 401, loss = 0.00043582
Iteration 402, loss = 0.00043555
Iteration 403, loss = 0.00043457
Iteration 404, loss = 0.00043523
Iteration 405, loss = 0.00043440
Iteration 406, loss = 0.00043423
Iteration 407, loss = 0.00043414
Iteration 408, loss = 0.00043358
Iteration 409, loss = 0.00043346
Iteration 410, loss = 0.00043374
Iteration 411, loss = 0.00043339
Iteration 412, loss = 0.00043293
Iteration 413, loss = 0.00043256
Iteration 414, loss = 0.00043258
Iteration 415, loss = 0.00043232
Iteration 416, loss = 0.00043211
Iteration 417, loss = 0.00043211
Iteration 418, loss = 0.00043111
Iteration 419, loss = 0.00043232
Iteration 420, loss = 0.00043191
Iteration 421, loss = 0.00043100
Iteration 422, loss = 0.00043123
Iteration 423, loss = 0.00043044
Iteration 424, loss = 0.00042986
Iteration 425, loss = 0.00043042
Iteration 426, loss = 0.00043075
Iteration 427, loss = 0.00043021
Iteration 428, loss = 0.00042941
Iteration 429, loss = 0.00042929
Iteration 430, loss = 0.00042903
Iteration 

Iteration 36, loss = 0.00605349
Iteration 37, loss = 0.00584204
Iteration 38, loss = 0.00565216
Iteration 39, loss = 0.00550427
Iteration 40, loss = 0.00538439
Iteration 41, loss = 0.00526161
Iteration 42, loss = 0.00515974
Iteration 43, loss = 0.00497030
Iteration 44, loss = 0.00500895
Iteration 45, loss = 0.00480938
Iteration 46, loss = 0.00479924
Iteration 47, loss = 0.00465698
Iteration 48, loss = 0.00459726
Iteration 49, loss = 0.00456044
Iteration 50, loss = 0.00454166
Iteration 51, loss = 0.00447891
Iteration 52, loss = 0.00438981
Iteration 53, loss = 0.00434276
Iteration 54, loss = 0.00434478
Iteration 55, loss = 0.00421714
Iteration 56, loss = 0.00424890
Iteration 57, loss = 0.00420012
Iteration 58, loss = 0.00415392
Iteration 59, loss = 0.00414411
Iteration 60, loss = 0.00410322
Iteration 61, loss = 0.00404949
Iteration 62, loss = 0.00402368
Iteration 63, loss = 0.00402136
Iteration 64, loss = 0.00396841
Iteration 65, loss = 0.00391342
Iteration 66, loss = 0.00393194
Iteratio

...

Iteration 148, loss = 0.00049143
Iteration 149, loss = 0.00048937
Iteration 150, loss = 0.00048928
Iteration 151, loss = 0.00048876
Iteration 152, loss = 0.00048783
Iteration 153, loss = 0.00048785
Iteration 154, loss = 0.00048691
Iteration 155, loss = 0.00048608
Iteration 156, loss = 0.00048574
Iteration 157, loss = 0.00048536
Iteration 158, loss = 0.00048445
Iteration 159, loss = 0.00048461
Iteration 160, loss = 0.00048375
Iteration 161, loss = 0.00048374
Iteration 162, loss = 0.00048249
Iteration 163, loss = 0.00048256
Iteration 164, loss = 0.00048118
Iteration 165, loss = 0.00048088
Iteration 166, loss = 0.00048072
Iteration 167, loss = 0.00047947
Iteration 168, loss = 0.00047955
Iteration 169, loss = 0.00047947
Iteration 170, loss = 0.00047893
Iteration 171, loss = 0.00047782
Iteration 172, loss = 0.00047803
Iteration 173, loss = 0.00047668
Iteration 174, loss = 0.00047660
Iteration 175, loss = 0.00047611
Iteration 176, loss = 0.00047540
Iteration 177, loss = 0.00047487
Iteration 

Iteration 300, loss = 0.00034226
Iteration 301, loss = 0.00034231
Iteration 302, loss = 0.00034217
Iteration 303, loss = 0.00034179
Iteration 304, loss = 0.00034177
Iteration 305, loss = 0.00034146
Iteration 306, loss = 0.00034149
Iteration 307, loss = 0.00034142
Iteration 308, loss = 0.00034094
Iteration 309, loss = 0.00034108
Iteration 310, loss = 0.00034091
Iteration 311, loss = 0.00034068
Iteration 312, loss = 0.00034062
Iteration 313, loss = 0.00034039
Iteration 314, loss = 0.00034016
Iteration 315, loss = 0.00034015
Iteration 316, loss = 0.00034001
Iteration 317, loss = 0.00033967
Iteration 318, loss = 0.00033957
Iteration 319, loss = 0.00033939
Iteration 320, loss = 0.00033932
Iteration 321, loss = 0.00033893
Iteration 322, loss = 0.00033877
Iteration 323, loss = 0.00033875
Iteration 324, loss = 0.00033866
Iteration 325, loss = 0.00033849
Iteration 326, loss = 0.00033832
Iteration 327, loss = 0.00033810
Iteration 328, loss = 0.00033805
Iteration 329, loss = 0.00033782
Iteration 

/Users/tpawlowski/workspace/dokstud/lazarou2020/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 401, loss = 0.00032746
Iteration 402, loss = 0.00032729
Iteration 403, loss = 0.00032690
Iteration 404, loss = 0.00032690
Iteration 405, loss = 0.00032681
Iteration 406, loss = 0.00032655
Iteration 407, loss = 0.00032641
Iteration 408, loss = 0.00032634
Iteration 409, loss = 0.00032632
Iteration 410, loss = 0.00032598
Iteration 411, loss = 0.00032578
Iteration 412, loss = 0.00032562
Iteration 413, loss = 0.00032586
Iteration 414, loss = 0.00032546
Iteration 415, loss = 0.00032546
Iteration 416, loss = 0.00032508
Iteration 417, loss = 0.00032506
Iteration 418, loss = 0.00032491
Iteration 419, loss = 0.00032473
Iteration 420, loss = 0.00032461
Iteration 421, loss = 0.00032444
Iteration 422, loss = 0.00032438
Iteration 423, loss = 0.00032418
Iteration 424, loss = 0.00032417
Iteration 425, loss = 0.00032387
Iteration 426, loss = 0.00032372
Iteration 427, loss = 0.00032369
Iteration 428, loss = 0.00032373
Iteration 429, loss = 0.00032332
Iteration 430, loss = 0.00032325
Iteration 

Iteration 650, loss = 0.00029653
Iteration 651, loss = 0.00029657
Iteration 652, loss = 0.00029635
Iteration 653, loss = 0.00029610
Iteration 654, loss = 0.00029627
Iteration 655, loss = 0.00029603
Iteration 656, loss = 0.00029597
Iteration 657, loss = 0.00029574
Iteration 658, loss = 0.00029561
Iteration 659, loss = 0.00029562
Iteration 660, loss = 0.00029553
Iteration 661, loss = 0.00029542
Iteration 662, loss = 0.00029520
Iteration 663, loss = 0.00029509
Iteration 664, loss = 0.00029495
Iteration 665, loss = 0.00029490
Iteration 666, loss = 0.00029487
Iteration 667, loss = 0.00029478
Iteration 668, loss = 0.00029458
Iteration 669, loss = 0.00029474
Iteration 670, loss = 0.00029437
Iteration 671, loss = 0.00029425
Iteration 672, loss = 0.00029417
Iteration 673, loss = 0.00029400
Iteration 674, loss = 0.00029409
Iteration 675, loss = 0.00029382
Iteration 676, loss = 0.00029376
error test: 5.136809610865824e-05, error train: 4.816775369258416e-05 training_time: 17215.6
saving ../final/

In [8]:
from IPython.display import HTML, display


html = f"<table>"
for label in ["test_error_orders_of_magnitude", "train_error_orders_of_magnitude", "test_error_absolute", "train_error_absolute"]:
    html += f"<tr><th colspan='{iterations+1}'>{label}</th></tr><tr><th>PCA</th>{''.join((f'<th>iteration{i}</th>') for i in range(iterations))}</tr>"
    for PCA_COMPONENTS in [8, 10, 11, 12, 13, 16, 20]:
        html += f"<tr><td>{PCA_COMPONENTS}</td>"
        for k in range(iterations):
            info_filename = f"../final/pca_models/MLPRegressor_{'_'.join(str(i) for i in hidden_layer_sizes)}_{PCA_COMPONENTS}_{LOWER_LIMIT}_{k}.json"
            f = open(info_filename)
            j = json.load(f)
            f.close()
            html += f"<td>{j[label]}</td>"
        html += "</tr>"
html += "</table>"
display(HTML(html))